## Defasagem Poder de Compra da remuneração do Servidores

In [1]:
# %pip install camelot-py[cv] tabulate
# !pip3 install pandas-ods-reader
# %pip install ezodf
# %pip install thefuzz
# %pip uninstall sqlalchemy-migrate
# %pip install camelot-py[cv] --use-pep517
# %pip install --upgrade camelot-py[cv]
# %pip install --upgrade PyPDF2
# %pip install pdfplumber

In [3]:
import os, re, requests
import pandas as pd
import camelot
import pdfplumber
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from IPython.display import clear_output


def download_pdf_files(url, save_folder):
    """Baixa todos os arquivos de "Relatório" de uma página da web e salva em uma pasta local.

    Args:
        url: A URL da página da web.
        save_folder: O caminho da pasta local onde os arquivos serão salvos.
    """    
    os.makedirs(save_folder, exist_ok=True)
    url = 'https://www.gov.br/servidor/pt-br/observatorio-de-pessoal-govbr/tabela-de-remuneracao-dos-servidores-publicos-federais-civis-e-dos-ex-territorios'
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
            print('Página acessada com sucesso, busando links para relatórios em PDF...')
    except requests.exceptions.Timeout:
        print(f"Tempo limite excedido ao tentar acessar a página: {url}")
    except requests.exceptions.RequestException as e:
        print(f"Ocorreu um erro ao baixar os arquivos, tente novamente mais tarde: {e}")

    soup = BeautifulSoup(response.content, 'html.parser')
    relatorio_links = [link['href'] for link in soup.find_all('a', string='Relatório')]
    print(f'{len(relatorio_links)} relatórios em PDF encontrados')

    pattern = re.compile(r'Caderno', re.IGNORECASE)  # Case-insensitive match
    caderno_links = [link['href'] for link in soup.find_all('a', string=pattern)]
    print(f'{len(caderno_links)} relatórios em PDF tipo caderno encontrados')

    print('Realizando downloads...')
    if relatorio_links:
        for link in tqdm(relatorio_links, desc="Obtendo relatórios..."):
            if link.endswith('@@download/file'):
                file_name = link.split('/')[-3].split('/')[0]
            else:
                file_name = link.split('/')[-1]
            file_path = os.path.join(save_folder, file_name)

            with requests.get(link, stream=True) as r:
                r.raise_for_status()
                with open(file_path, 'wb') as f:
                    for chunk in r.iter_content(chunk_size=8192):
                        f.write(chunk)
    else:
        print('Nenum link para relatórios em PDF encontrado')

    if caderno_links:   
        for link in tqdm(caderno_links, desc="Obtendo cadernos..."):
            if link.endswith('@@download/file'):
                file_name = link.split('/')[-3].split('/')[0]
            else:
                file_name = link.split('/')[-1]
            file_path = os.path.join(save_folder, file_name)

            with requests.get(link, stream=True) as r:
                r.raise_for_status()
                with open(file_path, 'wb') as f:
                    for chunk in r.iter_content(chunk_size=8192):
                        f.write(chunk)
    else:
        print('Nenhum link para cadernos em PDF encontrado')

    print(f"Relatórios PDF extraídos e salvos em {save_folder}")

    list_string =  [('tabela-de-remuneracao-','tab_'),('TabeladeRemunerao','tab_'),('TabelaRemunFev23Vol82Page','tab_82_2023'),
                    ('tabela-remunjul23-vol83-2','tab_83_2023'),('151217_tab_','tab_'),('2009-1','2009'),
                    ('-jan','_'),('_jan','_'),('_fev','_'),('_mar','_'),('-marco-','_'),('_abr','_'),('_mai','_'),
                    ('-maio','_'),('_jun','_'),('_jul','_'),('-julho-','_'),('_set','_'),('-ago-','_'),('-ago','_'),
                    ('_ago','_'),('_sto','_'),('-setembro','_'),('-set','_'),('_out','_'),('Nov','_'),('_dez','_'),
                    ('tab_82Page','tab_82_2023'),('tab_83-2','tab_83_2023'),('-semlogo22',''),('2012_03','2012'),
                    ('-semlogo22',''),('_eiro','_'),('-marco','_'),('_sto','_')]
    
    for _ in range(3):
        rename_pdf_file(save_folder, list_string)

def rename_pdf_file(save_folder, list_string):
    for filename in os.listdir(save_folder):
        if filename.endswith('.pdf') or filename.endswith('.ods'):  # Verifica se é um arquivo PDF ou ODS
            for i, j in list_string:
                new_filename = filename.replace(i, j)
                old_path = os.path.join(save_folder, filename)
                new_path = os.path.join(save_folder, new_filename)
                try:
                    os.rename(old_path, new_path)
                except OSError as e:
                    pass
                    # print(f"Error renaming: {e}")    

def download_ods_files(url, save_folder):
    """Baixa todos os arquivos .ods de uma página da web e salva em uma pasta local.

    Args:
        url: A URL da página da web.
        save_folder: O caminho da pasta local onde os arquivos serão salvos.
    """
    os.makedirs(save_folder, exist_ok=True)
    response = requests.get(url)
    if response.status_code == 200:
        print('Página acessada com sucesso, busando links para arquivos ODS...')

    response.raise_for_status()  # Verifica se a requisição foi bem-sucedida
    soup = BeautifulSoup(response.content, 'html.parser')
    ods_links = [link['href'] for link in soup.find_all('a', href=True) if link['href'].endswith('.ods')]
    if ods_links:
        print(f'{len(ods_links)} arquivos ODS encontrados, realizando downloads...')
    for link in ods_links:
        filename = os.path.basename(link)  # Obtém o nome do arquivo
        save_path = os.path.join(save_folder, filename)  # Caminho completo para salvar
        response = requests.get(link)
        response.raise_for_status()

        with open(save_path, 'wb') as file:
            file.write(response.content)

        print(f"Arquivo {filename} baixado com sucesso!")

def extract_summary_and_tables(pdf):
    """Extrai o sumário e tabelas do PDF."""
    li=None
    lf=None
    linhas=[]
    sumario = []
    all_tables = []

    for page in pdf.pages:
        print(f'Processando: {page.page_number:3}/{len(pdf.pages)}...')
        clear_output(wait=True)  # Limpa a saída e espera a próxima impressão
        text = page.extract_text()
        for n,line in enumerate(text.split('\n')):
            linhas.append(line)
            # localizar linha de final do sumário
            if 'Sumário' in line:
                li = n
            # localizar linha de início do sumário
            elif 'INDICE' in line:
                lf = n

        # Coletar tabelas
        for table in page.extract_tables():
            print(f'Tabela encontrada na página {page.page_number}')
            all_tables.append(table)
    
    print(f'Sumário das linhas {li} a {lf} total de {len(linhas)}')
    print(f'Total de {len(all_tables)} tabelas detectadas.')

    # Criar DataFrames
    if linhas:
        df_summary = pd.DataFrame(linhas, columns=['Sumário'])
    else:
        print('Nenhuma linha de sumário encontrada no arquivo PDF')
        df_summary = pd.DataFrame()
    if all_tables:
        df_tables = pd.DataFrame(all_tables[0])  # Criar DataFrame a partir da primeira tabela (assumindo que todas têm a mesma estrutura)
        for table in all_tables[1:]:
            df_tables = pd.concat([df_tables, pd.DataFrame(table)], ignore_index=True)
    else:
        print('Nenhuma tabela encontrada no arquivo PDF')
        df_tables = pd.DataFrame()

    return df_summary, li, lf, df_tables

def extract_remuneration_tables(pdf, df_summary):
    """Extrai as tabelas de remuneração com base no sumário."""
    all_data = []
    for _, row in df_summary.iterrows():
        orgao, cargo, paginas = row
        for pagina in map(int, paginas.split(',')):
            page = pdf.pages[pagina - 1]  # Páginas em pdfplumber começam em 0
            tables = page.extract_tables()

            # Encontrar a tabela de remuneração
            remuneration_table = None
            for table in tables:
                if any("CLASSE" in row[0] for row in table):
                    remuneration_table = table
                    break

        if remuneration_table is None:
            raise ValueError("Tabela de remuneração não encontrada no PDF.")

        # Extrair cabeçalho (considerando que a primeira linha contém os rótulos)
        header = remuneration_table[0]
        data = remuneration_table[1:]

        # Criar DataFrame
        df = pd.DataFrame(data, columns=header)

        # Extrair informações de órgão, cargo e posição do texto da página
        text = page.extract_text()
        orgao_cargo_matches = re.findall(r"(\d+)\.\s*(.+?)\s*-", text)  # Encontrar todos os órgãos e cargos

        # Criar dicionário para mapear número do órgão para nome do órgão e cargo
        orgao_cargo_map = {int(num): (orgao, cargo.split('-')[0].strip()) for num, orgao, cargo in orgao_cargo_matches}

        # Adicionar colunas de metadados
        df['Órgão'] = df['CLASSE'].astype(int).map(lambda x: orgao_cargo_map.get(x, ('', ''))[0])
        df['Cargo'] = df['CLASSE'].astype(int).map(lambda x: orgao_cargo_map.get(x, ('', ''))[1])

        # Limpeza e formatação
        df = df.drop(columns=["CLASSE"])  
        df = df.fillna('')  
        for col in ["ATIVO\nSUBSÍDIO (em R$)", "APOSENTADO\nSUBSÍDIO (em R$)"]:
            if col in df.columns:
                df[col] = df[col].astype(str).str.replace(".", "", regex=False).str.replace(",", ".", regex=False)
                df[col] = pd.to_numeric(df[col], errors='coerce')
                all_data.append(df)
    if all_data:  # Verifica se alguma tabela foi extraída
        return pd.concat(all_data, ignore_index=True)
    else:
        return pd.DataFrame()  # Retorna um DataFrame vazio se nenhuma tabela for encontrada

## Utilizando pdfplumber (conclui com sucesso mas não extrai bem os números)
def process_table(table, cargo, data_posicao):
    """Processa um DataFrame, adicionando informações de cargo e data."""
    table['Cargo'] = cargo
    table['Data Posição'] = data_posicao
    return table

def extract_tables_pdfplumber(file_path):
    """Extrai tabelas de um arquivo PDF usando pdfplumber."""
    with pdfplumber.open(file_path) as pdf:
        tables = []
        for page in pdf.pages:
            for table in page.extract_tables():
                df = pd.DataFrame(table)
                tables.append(df)
    return tables

def extract_remun_tables_pdfplumber(folder_path):
    """Extrai e consolida tabelas de remuneração de arquivos PDF na pasta."""
    all_data = []

    for filename in tqdm(os.listdir(folder_path), desc="Extraindo arquivos baixados..."):
        if filename.endswith('.pdf'):
            file_path = os.path.join(folder_path, filename)
            tables = extract_tables_pdfplumber(file_path)

            for table in tables:
                # Extrair informações de cargo e data (adaptar conforme o formato do arquivo)
                cargo = table.iloc[0, 0]  # Exemplo: assumindo que o cargo está na primeira célula
                data_posicao = filename.split('_')[0]  # Exemplo: assumindo que a data está no início do nome

                df = process_table(table, cargo, data_posicao)
                all_data.append(df)

    return pd.concat(all_data, ignore_index=True)

## Utilizando camelot ()
def process_table_camelot(table, cargo, data_posicao):
    """Processa um DataFrame (tabela do Camelot), adicionando informações de cargo e data."""
    df = table.df
    df['Cargo'] = cargo
    df['Data Posição'] = data_posicao
    return df

def extract_tables_camelot(file_path):
    """Extrai tabelas de um arquivo PDF usando Camelot.
    Parâmetros do Camelot:
        line_scale: aumentar o valor do parâmetro line_scale. Isso pode ajudar o Camelot a detectar as linhas horizontais que separam as células com quebras de linha.
        split_text: usar o parâmetro split_text=True. Isso fará com que o Camelot divida as células com quebras de linha em várias linhas.
        flag_size: usar o parâmetro flag_size=True. Isso pode ajudar o Camelot a identificar células com quebras de linha que são maiores do que o tamanho médio das células.
        strip_text: usar o parâmetro strip_text='\n'. Isso removerá as quebras de linha do texto extraído, o que pode ser útil se as quebras de linha não forem relevantes para a sua análise.
    """
    tables = camelot.read_pdf(file_path, pages='all', line_scale=40, split_text=True, flag_size=True)
    return tables


def split_merged_cells(table):
    """Divide células mescladas em um DataFrame do Camelot."""
    df = table.df
    for col in df.columns:
        df[col] = df[col].astype(str).str.split('\n')
        df = df.explode(col)
    df = df.fillna(method='ffill')
    return df

def extract_remun_tables_camelot(folder_path):
    """Extrai e consolida tabelas de remuneração de arquivos PDF na pasta (usando Camelot)."""
    all_data = []

    for filename in tqdm(sorted(os.listdir(folder_path)), desc="Extraindo arquivos baixados..."):
        print(f'Acessando: {filename}')
        if filename.endswith('.pdf'):
            file_path = os.path.join(folder_path, filename)
            try:
                tables = extract_tables_camelot(file_path)

                for table in tables:
                    # Pré-processamento com PyMuPDF (fitz) para dividir células mescladas
                    table.df = split_merged_cells(table)  

                    # Extrair informações de cargo e data (adaptar conforme o formato do arquivo)
                    cargo = table.df.iloc[0, 0]  # Corrigido para usar .iloc no DataFrame
                    data_posicao = filename.split('_')[0]

                    df = process_table_camelot(table, cargo, data_posicao)
                    all_data.append(df)
            except ValueError:
                print(f"Erro ao processar o arquivo {filename}. Pulando para o próximo.")

    return pd.concat(all_data, ignore_index=True)

def process_remuneration_data(df_remuneracao):
    """Processa o DataFrame para organizar as informações de remuneração."""

    # 1. Filtrar linhas relevantes (com valores numéricos)
    df_filtered = df_remuneracao[pd.to_numeric(df_remuneracao[9], errors='coerce').notnull()].copy()

    # 2. Extrair nível e parcelas (adaptar os índices conforme a estrutura do seu DataFrame)
    df_filtered['Nível'] = df_filtered[8].astype(str).str.extract(r'(P\d+)')
    df_filtered['Parcela'] = df_filtered[8]

    # 3. Pivotar o DataFrame
    df_pivot = df_filtered.pivot_table(index=['Cargo', 'Nível', 'Data Posição'], columns='Parcela', values=9)

    # Limpeza e formatação adicionais (opcional)
    df_pivot = df_pivot.replace('NaN', np.nan)  # Substituir 'NaN' por valores nulos
    df_pivot = df_pivot.applymap(lambda x: x.replace('.', '').replace(',', '.') if isinstance(x, str) else x)
    df_pivot = df_pivot.astype(float)

    return df_pivot

# Baixar relatórios em PDF com dados de remuneração

In [3]:
url = 'https://www.gov.br/servidor/pt-br/observatorio-de-pessoal-govbr/tabela-de-remuneracao-dos-servidores-publicos-federais-civis-e-dos-ex-territorios'
save_folder = 'F://TabRemunRel'

# Deixar comentado para evitar downloads desnecessário
download_pdf_files(url, save_folder)

Página acessada com sucesso, busando links para relatórios em PDF...
18 relatórios em PDF encontrados
56 relatórios em PDF tipo caderno encontrados
Realizando downloads...


Obtendo relatórios...:   0%|          | 0/18 [00:00<?, ?it/s]

Obtendo cadernos...:   0%|          | 0/56 [00:00<?, ?it/s]

Relatórios PDF extraídos e salvos em F://TabRemunRel


## Extrair sumários em cada relatório

In [66]:
def extract_cargo_and_page(lst_strings):
    """Extrai o cargo e o número da página de uma lista de strings.

    Args:
        strings: Uma lista de strings no formato "Cargo - ... - Nível ...... Página".

    Returns:
        Um dicionário onde as chaves são os nomes dos cargos e os valores são os números das páginas.
    """

    cargo_page_map = {}
    for string in lst_strings:
        match = re.search(r"(.+?)\s+\.{3,}\s+(\d+)", string)  # Regex para extrair cargo e página
        if match:
            cargo = match.group(1).strip()  # Remove espaços em branco do cargo
            page = int(match.group(2))  # Converte o número da página para inteiro
            cargo_page_map[cargo] = page
    return cargo_page_map

def starts_with_roman_numeral(string):
    """Verifica se uma string começa com um algarismo romano maiúsculo seguido de espaço, '-' ou '–' e espaço.

    Args:
        string: A string a ser verificada.

    Returns:
        True se a string começar com o padrão, False caso contrário.
    """

    pattern = r"^[IVXLCDM]+\s+[-–]\s+"  # Expressão regular com '-' e '–'
    return bool(re.match(pattern, string))

def extrair_sumario(folder_path, df_summary):
    all_data = []
    arquivos = sorted(os.listdir(folder_path))[-1:]
    print(f'{len(arquivos)} arquivos PDF a processar...')
    for filename in arquivos:
        print(f'\nProcessando {filename}...')
        if filename.endswith('.pdf'):
            file_path = os.path.join(folder_path, filename)
            with pdfplumber.open(file_path) as pdf:

                df_summary, li, lf, df_tables = extract_summary_and_tables(pdf)
                # df = extract_remuneration_tables(pdf, df_summary)
                # print(f'  {len(df.index)} linhas processadas')
    
    for n,i in enumerate([x for x in df_summary['Sumário']]):
        if i=='Sumário':
            li=n
        elif 'INDICE....' in i:
            lf=n

    ignorar = [str(x) for x in range(0,600)]
    lst_sumario = [x for x in df_summary['Sumário'].iloc[li:lf] if x not in ignorar]
    lst_tipo_remun = []
    lst_orgaos = []
    lst_cargos = []
    
    for i in lst_sumario:
        if i == 'Sumário':
            pass
        elif i.startswith('Elaborado'):
            periodo = i
        elif starts_with_roman_numeral(i):
            lst_tipo_remun.append(i)
        elif '....' not in i and i not in lst_tipo_remun and not i.startswith('Elaborado'):
            lst_orgaos.append(i)
        else:
            lst_cargos.append(i.split('  '))

    # map_cargo_page = extract_cargo_and_page(lst_cargos)

    return periodo, lst_sumario, lst_tipo_remun, lst_orgaos, lst_cargos

In [68]:
folder_path = 'F:\\TabRemunRel'
periodo, lst_sumario, lst_tipo_remun, lst_orgaos, lst_cargos = extrair_sumario(folder_path, df_summary)

Sumário das linhas 0 a 0 total de 64440
Total de 14 tabelas detectadas.


In [69]:
lst_orgaos

['01. Agência Brasileira de Inteligência - ABIN',
 '02. Agências Reguladoras - (ANA - ANAC - ANEEL - ANS - ANATEL - ANTAQ - ANTT - ANVISA - ANCINE - ANP)',
 'ANA - Agência Nacional de Águas e Saneamento Básico',
 'ANAC - Agência Nacional de Aviação Civil',
 'ANCINE - Agência Nacional do Cinema',
 'ANEEL- Agência Nacional de Energia Elétrica',
 'ANS - Agência Nacional de Saúde Suplementar',
 'ANP- Agência Nacional do Petróleo, Gás Natural e Biocombustíveis',
 'ANATEL - Agência Nacional de Telecomunicações',
 'ANTAQ - Agência Nacional de Transportes Aquaviários',
 'ANTT - Agência Nacional de Transportes Terrestres',
 'ANVISA - Agência Nacional de Vigilância Sanitária',
 '03. Área Jurídica',
 '04. Banco Central do Brasil',
 '05. Comissão de Valores Mobiliários - CVM',
 '06. Grupo Gestão',
 '07. Instituto de Pesquisa Econômica Aplicada - IPEA',
 '08. Ministério da Agricultura, Pecuária e Abastecimento - MAPA',
 '09. Polícia Federal',
 '10.Polícia Rodoviária',
 '11. Serviço Exterior Brasile

In [17]:
df_tables

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,None,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,83,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Nível Intermediário Posição: maio/2023,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ATIVO APOSENTADO\nCLASSE PADRÃO\nSUBSÍDIO (em ...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"lll 11.805,13 11.805,13\nESPECIAL ll 11.517,20...",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,FCE 2.05,FCE 3.05,FCE 4.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,FCE 2.04,FCE 3.04,FCE 4.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,FCE 2.03,FCE 3.03,FCE 4.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,FCE 2.02,FCE 3.02,FCE 4.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
[x for x in df_summary['Sumário']]

[]

In [19]:
[x for x in df_tables[0].iloc[2:] if x != '']

['Nível Intermediário Posição: maio/2023',
 'ATIVO APOSENTADO\nCLASSE PADRÃO\nSUBSÍDIO (em R$) SUBSÍDIO (em R$)',
 'lll 11.805,13 11.805,13\nESPECIAL ll 11.517,20 11.517,20\nl 11.236,30 11.236,30\nVl 10.701,25 10.701,25\nV 10.440,24 10.440,24\nlV 10.185,59 10.185,59\nPRIMEIRA\nlll 9.937,17 9.937,17\nll 9.694,79 9.694,79\nl 9.458,33 9.458,33\nVl 9.007,95 9.007,95\nV 8.788,22 8.788,22\nlV 8.573,87 8.573,87\nSEGUNDA\nlll 8.364,77 8.364,77\nll 8.160,75 8.160,75\nl 7.961,72 7.961,72\nV 7.582,57 7.582,57\nlV 7.397,64 7.397,64\nTERCEIRA lll 7.217,21 7.217,21\nll 7.041,18 7.041,18\nl 6.869,43 6.869,43',
 'CLASSES DE CAPACITAÇÃO I',
 'ATIVO e\nAPOSENTA\nVencimento DO ATIVO e APOSENTADO\nBásico (VB) SEM IQ - COM IQ - TOTAL (em R$)\nTOTAL\n(em R$)\nÁREA DE CONHECIMENTO COM RELAÇÃO DIRETA ÁREA DE CONHECIMENTO COM RELAÇÃO INDIRETA\n10% 15% 20% 25% 30% 52% 75% 10% 15% 20% 35% 50%\nA B=(A) C=A+ * D=A+* E=A+* F=A+* G=A+* H=A+* I=A+* J=A+* K=A+* L=A+ * M=A+* N=A+*',
 'CLASSES DE CAPACITAÇÃO III',
 'ATI

In [20]:
def extract_data_table(df_table):
    ignore = ['-','',None]
    for i in [x for x in df_table.iloc[2:] if (x not in ignore and type(x) is not float)]:
        print(f'{i}')
    print('-'*125)

In [16]:
df_tables

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,None,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,83,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Nível Intermediário Posição: maio/2023,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ATIVO APOSENTADO\nCLASSE PADRÃO\nSUBSÍDIO (em ...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"lll 11.805,13 11.805,13\nESPECIAL ll 11.517,20...",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,FCE 2.05,FCE 3.05,FCE 4.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,FCE 2.04,FCE 3.04,FCE 4.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,FCE 2.03,FCE 3.03,FCE 4.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,FCE 2.02,FCE 3.02,FCE 4.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
for i,series in enumerate(df_tables):
    extract_data_table(df_tables[i])

Nível Intermediário Posição: maio/2023
ATIVO APOSENTADO
CLASSE PADRÃO
SUBSÍDIO (em R$) SUBSÍDIO (em R$)
lll 11.805,13 11.805,13
ESPECIAL ll 11.517,20 11.517,20
l 11.236,30 11.236,30
Vl 10.701,25 10.701,25
V 10.440,24 10.440,24
lV 10.185,59 10.185,59
PRIMEIRA
lll 9.937,17 9.937,17
ll 9.694,79 9.694,79
l 9.458,33 9.458,33
Vl 9.007,95 9.007,95
V 8.788,22 8.788,22
lV 8.573,87 8.573,87
SEGUNDA
lll 8.364,77 8.364,77
ll 8.160,75 8.160,75
l 7.961,72 7.961,72
V 7.582,57 7.582,57
lV 7.397,64 7.397,64
TERCEIRA lll 7.217,21 7.217,21
ll 7.041,18 7.041,18
l 6.869,43 6.869,43
CLASSES DE CAPACITAÇÃO I
ATIVO e
APOSENTA
Vencimento DO ATIVO e APOSENTADO
Básico (VB) SEM IQ - COM IQ - TOTAL (em R$)
TOTAL
(em R$)
ÁREA DE CONHECIMENTO COM RELAÇÃO DIRETA ÁREA DE CONHECIMENTO COM RELAÇÃO INDIRETA
10% 15% 20% 25% 30% 52% 75% 10% 15% 20% 35% 50%
A B=(A) C=A+ * D=A+* E=A+* F=A+* G=A+* H=A+* I=A+* J=A+* K=A+* L=A+ * M=A+* N=A+*
CLASSES DE CAPACITAÇÃO III
ATIVO e
APOSENTA
Vencimento DO ATIVO e APOSENTADO
Básico (VB

In [35]:
import numpy as np
[x for x in df_tables[1].iloc[2:] if (x != '' and x is not None and x != float)]

['CLASSES DE CAPACITAÇÃO II\nATIVO e\nAPOSENTA\nVencimento DO ATIVO e APOSENTADO\nBásico (VB) SEM IQ - COM IQ - TOTAL (em R$)\nTOTAL\n(em R$)\nÁREA DE CONHECIMENTO COM RELAÇÃO DIRETA ÁREA DE CONHECIMENTO COM RELAÇÃO INDIRETA\n10% 15% 20% 25% 30% 52% 75% 10% 15% 20% 35% 50%\nA B=(A) C=A+ * D=A+* E=A+* F=A+* G=A+* H=A+* I=A+* J=A+* K=A+* L=A+ * M=A+* N=A+*',
 'CLASSES DE CAPACITAÇÃO IV\nATIVO e\nAPOSENTA\nVencimento DO ATIVO e APOSENTADO\nBásico (VB) SEM IQ - COM IQ - TOTAL (em R$)\nTOTAL\n(em R$)\nÁREA DE CONHECIMENTO COM RELAÇÃO DIRETA ÁREA DE CONHECIMENTO COM RELAÇÃO INDIRETA\n10% 15% 20% 25% 30% 52% 75% 10% 15% 20% 35% 50%\nA B=(A) C=A+ * D=A+* E=A+* F=A+* G=A+* H=A+* I=A+* J=A+* K=A+* L=A+ * M=A+* N=A+*',
 '9.838,59',
 '10.222,29',
 '10.620,97',
 '11.035,18',
 '11.465,56',
 '11.912,71',
 '12.377,31',
 '12.860,03',
 '13.361,57',
 '13.882,67',
 '14.424,09',
 '14.986,63',
 '15.571,11',
 '16.178,38',
 '16.809,34',
 '17.464,91',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,

In [10]:
df_summary[-180:-120]

,Sumário
5411,"Médico Cirurgião da Carreira da Previdência, d..."
5412,"Médico Cirurgião da Carreira da Previdência, d..."
5413,Médico da Área Médica do HFA - 20 h - NS 256
5414,Médico da Área Médica do HFA - 40 h - NS 256
5415,"Médico da Carreira da Previdência, da Saúde e ..."
5416,"Médico da Carreira da Previdência, da Saúde e ..."
5417,Médico da Carreira da Seguridade Social e do T...
5418,Médico da Carreira da Seguridade Social e do T...
5419,Médico da Carreira do Seguro Social de que tra...
5420,Médico da Carreira do Seguro Social de que tra...


In [66]:
df_summary[8]

,0,1,2
0,,P Poossiçiçããoo: :e essssaa i ninfoforrmmaaççã...,


In [65]:
df_summary[18]

,0,1
0,ATIVO APOSENTADO\nSUBSÍDIO (em R$) SUBSÍDIO (e...,None
1,None,"27.369,67 27.369,67"
2,None,"26.319,29 26.319,29"
3,None,"24.500,44 24.500,44"
4,None,"22.802,63 22.802,63"
5,None,"21.226,79 21.226,79"
6,None,"19.199,06 19.199,06"


In [43]:
df_temp = df_summary[9]
l1 = df_temp[0][0].split('\n')[0].split(' ')[:2]
l2 = df_temp[0][0].split('\n')[1].split('(em R$)')[:2]

In [55]:
for i in df_temp[1].values:
    if i is not None:
        print([float(x.replace('.','').replace(',','.')) for x in i.split(' ')])

[27303.7, 27303.7]
[24146.6, 24146.6]
[21014.49, 21014.49]


In [44]:
l1

['ATIVO', 'APOSENTADO']

In [45]:
l2

['SUBSÍDIO ', ' SUBSÍDIO ']

## Extrair tabelas de remuneração das carreiras

In [ ]:
folder_path = 'F:\\TabRemunRel'
df_remuneracao = extract_remun_tables_pdfplumber(folder_path)
df_remuneracao['Cargo']

In [ ]:
df_remuneracao[:60]

In [ ]:
# Camelot causando erro, Ghostscript instalado mas provavelmente não acessível
try:
    import ghostscript
    print("Ghostscript está instalado!")
except ImportError:
    print("Ghostscript não está instalado.")

In [ ]:
folder_path = 'F:\\TabRemunRel'
df_remuneracao = extract_remun_tables_camelot(folder_path)
df_remuneracao

In [ ]:
df_remuneracao[:60]

In [ ]:
import numpy as np
df_remuneracao_processado = process_remuneration_data(df_remuneracao)
df_remuneracao_processado

In [ ]:
df_remuneracao[60:120]

In [ ]:
import os
import pdfplumber
import pandas as pd
import re

def extract_remuneration_table(file_path):
    """Extrai a tabela de remuneração de um arquivo PDF."""
    with pdfplumber.open(file_path) as pdf:
        tables = []
        for page in pdf.pages:
            tables.extend(page.extract_tables())

        # Encontrar a tabela de remuneração
        remuneration_table = None
        for table in tables:
            if any("CLASSE" in row[0] for row in table):  # Verificar se "CLASSE" está presente em alguma célula da primeira coluna
                remuneration_table = table
                break

        if remuneration_table is None:
            raise ValueError("Tabela de remuneração não encontrada no PDF.")

        # Extrair cabeçalho (considerando que a primeira linha contém os rótulos)
        header = remuneration_table[0]
        data = remuneration_table[1:]

        # Criar DataFrame
        df = pd.DataFrame(data, columns=header)

        # Extrair informações de órgão, cargo e posição do texto da página
        text = page.extract_text()
        orgao_cargo_matches = re.findall(r"(\d+)\.\s*(.+?)\s*-", text)  # Encontrar todos os órgãos e cargos

        # Criar dicionário para mapear número do órgão para nome do órgão e cargo
        orgao_cargo_map = {int(num): (orgao, cargo.split('-')[0].strip()) for num, orgao, cargo in orgao_cargo_matches}

        # Adicionar colunas de metadados
        df['Órgão'] = df['CLASSE'].astype(int).map(lambda x: orgao_cargo_map.get(x, ('', ''))[0])
        df['Cargo'] = df['CLASSE'].astype(int).map(lambda x: orgao_cargo_map.get(x, ('', ''))[1])

        # Limpeza e formatação
        df = df.drop(columns=["CLASSE"])  
        df = df.fillna('')  
        for col in ["ATIVO\nSUBSÍDIO (em R$)", "APOSENTADO\nSUBSÍDIO (em R$)"]:
            if col in df.columns:
                df[col] = df[col].astype(str).str.replace(".", "", regex=False).str.replace(",", ".", regex=False)
                df[col] = pd.to_numeric(df[col], errors='coerce')

    return df


In [ ]:
all_data

In [ ]:
df_remuneracao = pd.concat(all_data, ignore_index=True)
df_remuneracao

In [ ]:
url = 'https://www.gov.br/servidor/pt-br/observatorio-de-pessoal-govbr/tabela-de-remuneracao-dos-servidores-publicos-federais-civis-e-dos-ex-territorios'
save_folder = 'F://TabRemun'

download_ods_files(url, save_folder)

In [ ]:
import os

os.listdir('F://TabRemun')

In [ ]:
list1 =  ['ESPECIAL', 'lll', 3703.72, 872.0, None, 752.0, 827.0, 902.0, 1462.0, 2925.0, None, 4575.72, 5327.72, 5402.72, 5477.72, 6037.72, 7500.72, None, 545.0, None, 4248.72, 5000.72, 5075.72, 5150.72, 5710.72, 7173.72, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
print(len(list1))

columns = ['PADRAO', 'NIVEL', 'A', 'B', 'VZ1' , 'C','D' ,'E', 'F','G', 'VZ2', 'H', 'I', 'J', 'VZ5', 'K', 'L', 'M', 'VZ3', 'N', 'VZ4', 'O', 'P', 'Q', 'R', 'S', 'T']
print(len(columns))

for n,i in enumerate(zip(columns,list1)):
    print(f'{n+1:2}: {i}')

In [ ]:
folder_path = 'C://TabRemun'
search_string = 'FIOCRUZ'

columns_to_extract = ['PADRAO', 'NIVEL', 'A', 'B', 'VZ1' , 'C','D' ,'E', 'F','G=(A+B)', 'VZ2', 'H=(A+B+D)', 'I=(A+B+E)', 'J=(A+B+F)', 'VZ5', 'K=(A+C)', 'L=(A+C+D)', 'M=(A+C+E)', 'VZ3', 'N=(A+C+F)', 'VZ4', 'O', 'P=(A+O)', 'Q=(A+D+O)', 'R=(A+E+O)', 'S=(A+F+O)']
print(len(columns_to_extract))

df_final = extract_data_from_zip(folder_path, search_string)

In [ ]:
import os
import zipfile
import numpy as np
import pandas as pd
import re
import ezodf
from io import BytesIO
import tempfile

def extract_data_from_zip(folder_path, search_string):
    all_data = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.zip'):
            zip_file_path = os.path.join(folder_path, filename)
            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                main_folder = zip_ref.namelist()[0]

                for item in zip_ref.namelist():
                    if item.startswith(main_folder) and search_string in item and item.endswith('/'):
                        subfolder_name = item
                        break
                else:
                    print(f"Subpasta com '{search_string}' não encontrada em {filename}.")
                    continue

                for name in zip_ref.namelist():
                    if name.startswith(subfolder_name) and name.endswith('.ods'):
                        with zip_ref.open(name) as file:
                            try:
                                ods_data = BytesIO(file.read())

                                with tempfile.NamedTemporaryFile(delete=False, suffix='.ods') as temp:
                                    temp.write(ods_data.getvalue())
                                    temp_path = temp.name

                                doc = ezodf.opendoc(temp_path)
                                sheet = doc.sheets[0]
                                # Converte a planilha em um DataFrame do Pandas
                                data = []
                                for i, row in enumerate(sheet.rows()):
                                    data.append([cell.value for cell in row])

                                # Cria um DataFrame
                                df = pd.DataFrame(data)
                                os.remove(temp_path)

                                # Encontra a data "Posição" iterando de baixo para cima
                                date = None
                                for index, row in df[::-1].iterrows():
                                    for cell in row:
                                        if cell is not None:  # Verifica se a célula não é None
                                            date_string = re.search(r'(janeiro|fevereiro|março|abril|maio|junho|julho|agosto|setembro|outubro|novembro|dezembro)/\d{4}', str(cell))
                                            if date_string:
                                                date = date_string.group()
                                                break
                                    if date:
                                        break

                                # Normaliza os nomes das colunas (remove espaços, quebras de linha, converte para minúsculas e substitui espaços por underscores)
                                df.columns = (
                                    df.iloc[0]
                                    .astype(str)
                                    .str.strip()
                                    .str.replace('\n', '', regex=False)
                                    .str.replace(r'[ =+\(\)]', '', regex=True)  # Remove caracteres especiais
                                    .str.lower()
                                )

                                ['PADRAO', 'NIVEL', 'A', 'B', 'VZ1' , 'C','D' ,'E', 'F','G', 'VZ2', 'H', 'I', 'J', 'VZ5', 'K', 'L', 'M', 'VZ3', 'N', 'VZ4', 'O', 'P', 'Q', 'R', 'S', 'T']

                                # Mapeia os nomes das colunas para os nomes corretos (incluindo "Classe")
                                column_mapping = {
                                    'Cargo': 'Cargo',
                                    'Classe': 'Classe',
                                    'Padrao': 'Padrao',
                                    'A': 'vb',
                                    'B': 'ativo_100p_sem_rt',
                                    'C': 'ativo_100p_aperf_espec',
                                    'D': 'ativo_100p_mestre',
                                    'E': 'ativo_100p_doutor',
                                    'F': 'aposentado_sem_rt',
                                    'G': 'aposentado_aperf_espec',
                                    'H': 'aposentado_mestre',
                                    'I': 'aposentado_doutor',
                                    'K': 'ativo_100p_sem_rt',
                                    'L': 'ativo_100p_aperf_espec',
                                    'M': 'ativo_100p_mestre',
                                    'N': 'ativo_100p_doutor',
                                    'O': 'aposentado_sem_rt',
                                    'P': 'aposentado_aperf_espec',
                                    'Q': 'aposentado_mestre',
                                    'R': 'aposentado_doutor',
                                    'T': 'aposentado_doutor',
                                }

                                df = df.rename(columns=column_mapping)
                                df = df.iloc[1:].copy()

                                # Remove colunas duplicadas antes de usar o reindex
                                df = df.loc[:,~df.columns.duplicated()]
                                # Extrai apenas as colunas desejadas, preenchendo com NaN se não existirem
                                df_extracted = df.reindex(columns=['Cargo','Classe', 'Padrao', 'VB', 'Ativo_100p_semRT', 'Ativo_100p_AperfEspec', 'Ativo_100p_Mestre', 'Ativo_100p_Doutor', 'Aposentado_SemRT', 'Aposentado_AperfEspec', 'Aposentado_Mestre', 'Aposentado_Doutor', 'Data'], fill_value=np.nan)
                                df_extracted.loc[:, 'Data'] = date  # Use .loc para atribuir o valor

                                # Extrai o nome do cargo do nome do arquivo
                                cargo = os.path.splitext(os.path.basename(name))[0]
                                cargo = re.sub(r'[0-9]', '', cargo).strip()
                                df_extracted['Cargo'] = cargo
                                # Converte todas as colunas para numérico, se possível, exceto 'Data' e 'Cargo'
                                for col in df_extracted.columns:
                                    if col not in ['Data', 'Cargo', 'classe', 'padrao']:
                                        df_extracted[col] = pd.to_numeric(df_extracted[col], errors='coerce')

                                # Remove linhas com todos os valores nulos
                                df_extracted = df_extracted.dropna(how='all')

                                # Extrai a classe e o padrão para cada linha
                                classe = None
                                padrao = None
                                for idx, row in df_extracted.iterrows():  
                                    if pd.notna(row.get('classe', None)):  # Verifica se a coluna 'classe' existe
                                        classe = row['classe']
                                    else:
                                        classe = 'NA'  # Atribui 'NA' se a coluna não existir

                                    if pd.notna(row.get('padrao', None)):  # Verifica se a coluna 'padrao' existe
                                        padrao = row['padrao']

                                    # Adiciona os dados apenas se classe e padrão não forem nulos
                                    if classe is not None and padrao is not None:
                                        df_extracted.loc[idx, 'CLASSE'] = classe
                                        df_extracted.loc[idx, 'PADRÃO'] = padrao

                                all_data.append(df_extracted)
                            except Exception as e:
                                print(f"Erro ao ler a planilha {name}: {e}")

    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        return final_df
    else:
        return None


In [ ]:
# Download arquivos de dados zipados
# url = 'https://www.gov.br/servidor/pt-br/observatorio-de-pessoal-govbr/tabela-de-remuneracao-dos-servidores-publicos-federais-civis-e-dos-ex-territorios'
# save_folder = 'C://TabRemun'

# download_ods_files(url, save_folder)

In [ ]:
import os
import zipfile
import pandas as pd
import re
import tempfile
import numpy as np

def extract_data_from_zip(folder_path, search_string):
    all_data = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.zip'):
            zip_file_path = os.path.join(folder_path, filename)
            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                main_folder = zip_ref.namelist()[0]

                for item in zip_ref.namelist():
                    if item.startswith(main_folder) and search_string in item and item.endswith('/'):
                        subfolder_name = item
                        break
                else:
                    print(f"Subpasta com '{search_string}' não encontrada em {filename}.")
                    continue

                for name in zip_ref.namelist():
                    if name.startswith(subfolder_name) and name.endswith('.ods'):
                        with zip_ref.open(name) as file:
                            try:
                                text = file.read().decode('utf-8', errors='replace')

                                # Find the header row
                                header_row = None
                                for i, line in enumerate(text.splitlines()):
                                    if 'Padrão' in line:
                                        header_row = i
                                        break

                                if header_row is None:
                                    raise ValueError("Rótulos das colunas não encontrados.")

                                # Extract data starting from the row after the header
                                data = []
                                for line in text.splitlines()[header_row + 1:]:
                                    row_values = [x.strip() for x in line.split('\t') if x.strip()]
                                    data.append(row_values)

                                # Create DataFrame
                                columns = ['PADRAO', 'NIVEL', 'A', 'B', 'VZ1' , 'C','D' ,'E', 'F','G', 'VZ2', 'H', 'I', 'J', 'VZ5', 'K', 'L', 'M', 'VZ3', 'N', 'VZ4', 'O', 'P', 'Q', 'R', 'S', 'T']
                                df = pd.DataFrame(data, columns=columns[:len(data[0])])  # Use the number of columns in the data

                                # Encontra a data "Posição" iterando de baixo para cima
                                date = None
                                for index, row in df[::-1].iterrows():
                                    for cell in row:
                                        if cell is not None:  # Verifica se a célula não é None
                                            date_string = re.search(r'(janeiro|fevereiro|março|abril|maio|junho|julho|agosto|setembro|outubro|novembro|dezembro)/\d{4}', str(cell))
                                            if date_string:
                                                date = date_string.group()
                                                break
                                    if date:
                                        break

                                # Normalize column names
                                df.columns = (
                                    df.columns
                                    .astype(str)
                                    .str.strip()
                                    .str.replace('\n', '', regex=False)
                                    .str.replace(r'[ =+\(\)]', '', regex=True)
                                    .str.lower()
                                )

                                # Map column names
                                column_mapping = {
                                    'a': 'vb',
                                    'k': 'ativo_100p_sem_rt',
                                    'l': 'ativo_100p_aperf_espec',
                                    'm': 'ativo_100p_mestre',
                                    'n': 'ativo_100p_doutor',
                                    'p': 'aposentado_sem_rt',
                                    'q': 'aposentado_aperf_espec',
                                    'r': 'aposentado_mestre',
                                    's': 'aposentado_doutor',
                                    'classe': 'classe',
                                    'padrao': 'padrao',
                                    'ativo 100% sem rt': 'ativo_100p_sem_rt',
                                    'ativo 100% aperfeiçoamento/especialização': 'ativo_100p_aperf_espec',
                                    'ativo 100% mestre': 'ativo_100p_mestre',
                                    'ativo 100% doutor': 'ativo_100p_doutor',
                                    'aposentado sem rt': 'aposentado_sem_rt',
                                    'aposentado aperfeiçoamento/especialização': 'aposentado_aperf_espec',
                                    'aposentado mestre': 'aposentado_mestre',
                                    'aposentado doutor': 'aposentado_doutor',
                                }
                                df = df.rename(columns=column_mapping)

                                # Extract relevant columns, fill missing columns with NaN values
                                df_extracted = df.reindex(columns=['classe', 'padrao', 'vb', 'ativo_100p_sem_rt', 'ativo_100p_aperf_espec', 'ativo_100p_mestre', 'ativo_100p_doutor', 'aposentado_sem_rt', 'aposentado_aperf_espec', 'aposentado_mestre', 'aposentado_doutor', 'Data', 'Cargo'], fill_value=np.nan)

                                # Extract cargo from file name
                                cargo = os.path.splitext(os.path.basename(name))[0]
                                cargo = re.sub(r'[0-9]', '', cargo).strip()

                                df_extracted['Data'] = date
                                df_extracted['Cargo'] = cargo

                                # Convert numeric columns to numeric data types
                                for col in df_extracted.columns:
                                    if col not in ['Data', 'Cargo', 'classe', 'padrao']:
                                        df_extracted[col] = pd.to_numeric(df_extracted[col], errors='coerce')

                                # Remove rows with all NaN values
                                df_extracted = df_extracted.dropna(how='all')

                                # Extract 'classe' and 'padrao' for each row, filling with 'NA' if not found
                                classe = None
                                padrao = None
                                for idx, row in df_extracted.iterrows():
                                    if pd.notna(row.get('classe', None)):
                                        classe = row['classe']
                                    else:
                                        classe = 'NA'

                                    if pd.notna(row.get('padrao', None)):
                                        padrao = row['padrao']

                                    # Add the data only if classe and padrao are not None
                                    if classe is not None and padrao is not None:
                                        df_extracted.loc[idx, 'CLASSE'] = classe
                                        df_extracted.loc[idx, 'PADRÃO'] = padrao

                                all_data.append(df_extracted)
                            except Exception as e:
                                print(f"Erro ao ler a planilha {name}: {e}")

    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        return final_df
    else:
        return None

In [ ]:
folder_path = 'C://TabRemun'
search_string = 'FIOCRUZ'
df_final = extract_data_from_zip(folder_path, search_string)

In [ ]:
df_final.keys()

In [ ]:
[x for x in df_final['Data'].unique()]

In [ ]:
[x for x in df_final['Cargo'].unique()]

In [ ]:
df_final[:60]

In [ ]:
# Exemplo de uso
folder_path = 'F://TabRemun'
search_string = 'FIOCRUZ'
columns_to_extract = ["CLASSE", "PADRÃO", "A","B","C","D","E","F","G=(A+B)","H=(A+B+D)","I=(A+B+E)","J=(A+B+F)","K=(A+C)", "L=(A+C+D)", "M=(A+C+E)", "N=(A+C+F)",	"O", "P=(A+O)",	"Q=(A+D+O)", "R=(A+E+O)", "S=(A+F+O)"]
rotulos = {"VB": "A",
           "Teto Atividade Sem RT": "K=(A+C)",
           "Teto Atividade Aperf./Espec.":	"L=(A+C+D)",
           "Teto Atividade Mestre": "M=(A+C+E)",
           "Teto Atividade Doutor": "N=(A+C+F)",
           "Teto Aposentado Sem RT": "P=(A+O)",
           "Teto Aposentado Aperf./Espec.":	"Q=(A+D+O)",
           "Teto Aposentado Mestre": "R=(A+E+O)",
           "Teto Aposentado Doutor": "S=(A+F+O)",
           }

df_final = extract_data_from_zip(folder_path, search_string, columns_to_extract)

if df_final is not None:
    print(df_final)
else:
    print("Nenhum dado encontrado.")

In [ ]:
url_ipca_completo = "https://servicodados.ibge.gov.br/api/v3/agregados/1737/periodos/197912|198001|198002|198003|198004|198005|198006|198007|198008|198009|198010|198011|198012|198101|198102|198103|198104|198105|198106|198107|198108|198109|198110|198111|198112|198201|198202|198203|198204|198205|198206|198207|198208|198209|198210|198211|198212|198301|198302|198303|198304|198305|198306|198307|198308|198309|198310|198311|198312|198401|198402|198403|198404|198405|198406|198407|198408|198409|198410|198411|198412|198501|198502|198503|198504|198505|198506|198507|198508|198509|198510|198511|198512|198601|198602|198603|198604|198605|198606|198607|198608|198609|198610|198611|198612|198701|198702|198703|198704|198705|198706|198707|198708|198709|198710|198711|198712|198801|198802|198803|198804|198805|198806|198807|198808|198809|198810|198811|198812|198901|198902|198903|198904|198905|198906|198907|198908|198909|198910|198911|198912|199001|199002|199003|199004|199005|199006|199007|199008|199009|199010|199011|199012|199101|199102|199103|199104|199105|199106|199107|199108|199109|199110|199111|199112|199201|199202|199203|199204|199205|199206|199207|199208|199209|199210|199211|199212|199301|199302|199303|199304|199305|199306|199307|199308|199309|199310|199311|199312|199401|199402|199403|199404|199405|199406|199407|199408|199409|199410|199411|199412|199501|199502|199503|199504|199505|199506|199507|199508|199509|199510|199511|199512|199601|199602|199603|199604|199605|199606|199607|199608|199609|199610|199611|199612|199701|199702|199703|199704|199705|199706|199707|199708|199709|199710|199711|199712|199801|199802|199803|199804|199805|199806|199807|199808|199809|199810|199811|199812|199901|199902|199903|199904|199905|199906|199907|199908|199909|199910|199911|199912|200001|200002|200003|200004|200005|200006|200007|200008|200009|200010|200011|200012|200101|200102|200103|200104|200105|200106|200107|200108|200109|200110|200111|200112|200201|200202|200203|200204|200205|200206|200207|200208|200209|200210|200211|200212|200301|200302|200303|200304|200305|200306|200307|200308|200309|200310|200311|200312|200401|200402|200403|200404|200405|200406|200407|200408|200409|200410|200411|200412|200501|200502|200503|200504|200505|200506|200507|200508|200509|200510|200511|200512|200601|200602|200603|200604|200605|200606|200607|200608|200609|200610|200611|200612|200701|200702|200703|200704|200705|200706|200707|200708|200709|200710|200711|200712|200801|200802|200803|200804|200805|200806|200807|200808|200809|200810|200811|200812|200901|200902|200903|200904|200905|200906|200907|200908|200909|200910|200911|200912|201001|201002|201003|201004|201005|201006|201007|201008|201009|201010|201011|201012|201101|201102|201103|201104|201105|201106|201107|201108|201109|201110|201111|201112|201201|201202|201203|201204|201205|201206|201207|201208|201209|201210|201211|201212|201301|201302|201303|201304|201305|201306|201307|201308|201309|201310|201311|201312|201401|201402|201403|201404|201405|201406|201407|201408|201409|201410|201411|201412|201501|201502|201503|201504|201505|201506|201507|201508|201509|201510|201511|201512|201601|201602|201603|201604|201605|201606|201607|201608|201609|201610|201611|201612|201701|201702|201703|201704|201705|201706|201707|201708|201709|201710|201711|201712|201801|201802|201803|201804|201805|201806|201807|201808|201809|201810|201811|201812|201901|201902|201903|201904|201905|201906|201907|201908|201909|201910|201911|201912|202001|202002|202003|202004|202005|202006|202007|202008|202009|202010|202011|202012|202101|202102|202103|202104|202105|202106|202107|202108|202109|202110|202111|202112|202201|202202|202203|202204|202205|202206|202207|202208|202209|202210|202211|202212|202301|202302|202303|202304|202305|202306|202307|202308|202309|202310|202311|202312|202401|202402|202403|202404|202405/variaveis/2266?localidades=N1[all]"
url_ipca = "https://servicodados.ibge.gov.br/api/v3/agregados/1737/periodos/199212|199301|199302|199303|199304|199305|199306|199307|199308|199309|199310|199311|199312|199401|199402|199403|199404|199405|199406|199407|199408|199409|199410|199411|199412|199501|199502|199503|199504|199505|199506|199507|199508|199509|199510|199511|199512|199601|199602|199603|199604|199605|199606|199607|199608|199609|199610|199611|199612|199701|199702|199703|199704|199705|199706|199707|199708|199709|199710|199711|199712|199801|199802|199803|199804|199805|199806|199807|199808|199809|199810|199811|199812|199901|199902|199903|199904|199905|199906|199907|199908|199909|199910|199911|199912|200001|200002|200003|200004|200005|200006|200007|200008|200009|200010|200011|200012|200101|200102|200103|200104|200105|200106|200107|200108|200109|200110|200111|200112|200201|200202|200203|200204|200205|200206|200207|200208|200209|200210|200211|200212|200301|200302|200303|200304|200305|200306|200307|200308|200309|200310|200311|200312|200401|200402|200403|200404|200405|200406|200407|200408|200409|200410|200411|200412|200501|200502|200503|200504|200505|200506|200507|200508|200509|200510|200511|200512|200601|200602|200603|200604|200605|200606|200607|200608|200609|200610|200611|200612|200701|200702|200703|200704|200705|200706|200707|200708|200709|200710|200711|200712|200801|200802|200803|200804|200805|200806|200807|200808|200809|200810|200811|200812|200901|200902|200903|200904|200905|200906|200907|200908|200909|200910|200911|200912|201001|201002|201003|201004|201005|201006|201007|201008|201009|201010|201011|201012|201101|201102|201103|201104|201105|201106|201107|201108|201109|201110|201111|201112|201201|201202|201203|201204|201205|201206|201207|201208|201209|201210|201211|201212|201301|201302|201303|201304|201305|201306|201307|201308|201309|201310|201311|201312|201401|201402|201403|201404|201405|201406|201407|201408|201409|201410|201411|201412|201501|201502|201503|201504|201505|201506|201507|201508|201509|201510|201511|201512|201601|201602|201603|201604|201605|201606|201607|201608|201609|201610|201611|201612|201701|201702|201703|201704|201705|201706|201707|201708|201709|201710|201711|201712|201801|201802|201803|201804|201805|201806|201807|201808|201809|201810|201811|201812|201901|201902|201903|201904|201905|201906|201907|201908|201909|201910|201911|201912|202001|202002|202003|202004|202005|202006|202007|202008|202009|202010|202011|202012|202101|202102|202103|202104|202105|202106|202107|202108|202109|202110|202111|202112|202201|202202|202203|202204|202205|202206|202207|202208|202209|202210|202211|202212|202301|202302|202303|202304|202305|202306|202307|202308|202309|202310|202311|202312|202401|202402|202403|202404|202405/variaveis/2266?localidades=N1[all]"
# url_tab_var = "https://apisidra.ibge.gov.br/values/t/1737/n1/all/v/2266/p/all"
indices_mensais = {}

response = requests.get(url_ipca_completo)
if response.status_code == 200:
    print('Requisição realizada com sucesso!')
response.raise_for_status()

dados_ipca = json.loads(response.content)
indices_ipca = dados_ipca[0]['resultados'][0].get('series')[0].get('serie')
print(f'{len(indices_ipca)} índices IPCA mensais extraídos do IBGE')

# Cria uma lista de tuplas a partir dos itens do dicionário
lista_dados = list(indices_ipca.items())

# Cria o DataFrame
df = pd.DataFrame(lista_dados, columns=["AnoMes", "Valor"])

In [ ]:
df

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Dados fornecidos
anos = list(range(2010, 2027))
inflacao = [0.02, 0.015, 0.025, 0.029, 0.018, 0.02, 0.015, 0.025, 0.029, 0.018, 0.012, 0.047, 0.07, 0.05, 0.03, 0.025, 0.02]
ajuste = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.2, 0.2, 0.2]

# Calcula os valores reais ajustados
valores_reais = [100]  # Começa com 100 em 2015
for i in range(1, len(anos)):
    valor_anterior = valores_reais[i - 1]
    valor_atual = valor_anterior * (1 - inflacao[i - 1]) * (1 + ajuste[i - 1])
    valores_reais.append(valor_atual)

# Cria um DataFrame
df = pd.DataFrame({'Anos': anos, 'Valores em Reais': valores_reais})

# Cria o gráfico de barras com Plotly
fig = go.Figure(data=[go.Bar(x=df['Anos'], y=df['Valores em Reais'])])

# Personaliza o layout
fig.update_layout(
    title='Poder de Compra Real Ajustado (2010-2026)',
    xaxis_title='Ano',
    yaxis_title='Valor Real (R$)',
    xaxis=dict(
        dtick=1,  # Garante que cada ano seja mostrado
        tickmode='linear'  # Modo linear para ticks
    )
)

# Atualiza o gráfico para adicionar rótulos de dados
fig.update_traces(texttemplate='%{y:.2f}', textposition='inside')

# Exibe o gráfico
fig.show()


In [ ]:
# dados_ipca

In [ ]:
def plotar_perdas(ano_inicio, ano_final, reajustes_anuais):
    url_ipca_completo = "https://servicodados.ibge.gov.br/api/v3/agregados/1737/periodos/197912|198001|198002|198003|198004|198005|198006|198007|198008|198009|198010|198011|198012|198101|198102|198103|198104|198105|198106|198107|198108|198109|198110|198111|198112|198201|198202|198203|198204|198205|198206|198207|198208|198209|198210|198211|198212|198301|198302|198303|198304|198305|198306|198307|198308|198309|198310|198311|198312|198401|198402|198403|198404|198405|198406|198407|198408|198409|198410|198411|198412|198501|198502|198503|198504|198505|198506|198507|198508|198509|198510|198511|198512|198601|198602|198603|198604|198605|198606|198607|198608|198609|198610|198611|198612|198701|198702|198703|198704|198705|198706|198707|198708|198709|198710|198711|198712|198801|198802|198803|198804|198805|198806|198807|198808|198809|198810|198811|198812|198901|198902|198903|198904|198905|198906|198907|198908|198909|198910|198911|198912|199001|199002|199003|199004|199005|199006|199007|199008|199009|199010|199011|199012|199101|199102|199103|199104|199105|199106|199107|199108|199109|199110|199111|199112|199201|199202|199203|199204|199205|199206|199207|199208|199209|199210|199211|199212|199301|199302|199303|199304|199305|199306|199307|199308|199309|199310|199311|199312|199401|199402|199403|199404|199405|199406|199407|199408|199409|199410|199411|199412|199501|199502|199503|199504|199505|199506|199507|199508|199509|199510|199511|199512|199601|199602|199603|199604|199605|199606|199607|199608|199609|199610|199611|199612|199701|199702|199703|199704|199705|199706|199707|199708|199709|199710|199711|199712|199801|199802|199803|199804|199805|199806|199807|199808|199809|199810|199811|199812|199901|199902|199903|199904|199905|199906|199907|199908|199909|199910|199911|199912|200001|200002|200003|200004|200005|200006|200007|200008|200009|200010|200011|200012|200101|200102|200103|200104|200105|200106|200107|200108|200109|200110|200111|200112|200201|200202|200203|200204|200205|200206|200207|200208|200209|200210|200211|200212|200301|200302|200303|200304|200305|200306|200307|200308|200309|200310|200311|200312|200401|200402|200403|200404|200405|200406|200407|200408|200409|200410|200411|200412|200501|200502|200503|200504|200505|200506|200507|200508|200509|200510|200511|200512|200601|200602|200603|200604|200605|200606|200607|200608|200609|200610|200611|200612|200701|200702|200703|200704|200705|200706|200707|200708|200709|200710|200711|200712|200801|200802|200803|200804|200805|200806|200807|200808|200809|200810|200811|200812|200901|200902|200903|200904|200905|200906|200907|200908|200909|200910|200911|200912|201001|201002|201003|201004|201005|201006|201007|201008|201009|201010|201011|201012|201101|201102|201103|201104|201105|201106|201107|201108|201109|201110|201111|201112|201201|201202|201203|201204|201205|201206|201207|201208|201209|201210|201211|201212|201301|201302|201303|201304|201305|201306|201307|201308|201309|201310|201311|201312|201401|201402|201403|201404|201405|201406|201407|201408|201409|201410|201411|201412|201501|201502|201503|201504|201505|201506|201507|201508|201509|201510|201511|201512|201601|201602|201603|201604|201605|201606|201607|201608|201609|201610|201611|201612|201701|201702|201703|201704|201705|201706|201707|201708|201709|201710|201711|201712|201801|201802|201803|201804|201805|201806|201807|201808|201809|201810|201811|201812|201901|201902|201903|201904|201905|201906|201907|201908|201909|201910|201911|201912|202001|202002|202003|202004|202005|202006|202007|202008|202009|202010|202011|202012|202101|202102|202103|202104|202105|202106|202107|202108|202109|202110|202111|202112|202201|202202|202203|202204|202205|202206|202207|202208|202209|202210|202211|202212|202301|202302|202303|202304|202305|202306|202307|202308|202309|202310|202311|202312|202401|202402|202403|202404|202405/variaveis/2266?localidades=N1[all]"
    url_ipca = "https://servicodados.ibge.gov.br/api/v3/agregados/1737/periodos/199212|199301|199302|199303|199304|199305|199306|199307|199308|199309|199310|199311|199312|199401|199402|199403|199404|199405|199406|199407|199408|199409|199410|199411|199412|199501|199502|199503|199504|199505|199506|199507|199508|199509|199510|199511|199512|199601|199602|199603|199604|199605|199606|199607|199608|199609|199610|199611|199612|199701|199702|199703|199704|199705|199706|199707|199708|199709|199710|199711|199712|199801|199802|199803|199804|199805|199806|199807|199808|199809|199810|199811|199812|199901|199902|199903|199904|199905|199906|199907|199908|199909|199910|199911|199912|200001|200002|200003|200004|200005|200006|200007|200008|200009|200010|200011|200012|200101|200102|200103|200104|200105|200106|200107|200108|200109|200110|200111|200112|200201|200202|200203|200204|200205|200206|200207|200208|200209|200210|200211|200212|200301|200302|200303|200304|200305|200306|200307|200308|200309|200310|200311|200312|200401|200402|200403|200404|200405|200406|200407|200408|200409|200410|200411|200412|200501|200502|200503|200504|200505|200506|200507|200508|200509|200510|200511|200512|200601|200602|200603|200604|200605|200606|200607|200608|200609|200610|200611|200612|200701|200702|200703|200704|200705|200706|200707|200708|200709|200710|200711|200712|200801|200802|200803|200804|200805|200806|200807|200808|200809|200810|200811|200812|200901|200902|200903|200904|200905|200906|200907|200908|200909|200910|200911|200912|201001|201002|201003|201004|201005|201006|201007|201008|201009|201010|201011|201012|201101|201102|201103|201104|201105|201106|201107|201108|201109|201110|201111|201112|201201|201202|201203|201204|201205|201206|201207|201208|201209|201210|201211|201212|201301|201302|201303|201304|201305|201306|201307|201308|201309|201310|201311|201312|201401|201402|201403|201404|201405|201406|201407|201408|201409|201410|201411|201412|201501|201502|201503|201504|201505|201506|201507|201508|201509|201510|201511|201512|201601|201602|201603|201604|201605|201606|201607|201608|201609|201610|201611|201612|201701|201702|201703|201704|201705|201706|201707|201708|201709|201710|201711|201712|201801|201802|201803|201804|201805|201806|201807|201808|201809|201810|201811|201812|201901|201902|201903|201904|201905|201906|201907|201908|201909|201910|201911|201912|202001|202002|202003|202004|202005|202006|202007|202008|202009|202010|202011|202012|202101|202102|202103|202104|202105|202106|202107|202108|202109|202110|202111|202112|202201|202202|202203|202204|202205|202206|202207|202208|202209|202210|202211|202212|202301|202302|202303|202304|202305|202306|202307|202308|202309|202310|202311|202312|202401|202402|202403|202404|202405/variaveis/2266?localidades=N1[all]"
    # url_tab_var = "https://apisidra.ibge.gov.br/values/t/1737/n1/all/v/2266/p/all"
    indices_mensais = {}

    response = requests.get(url_ipca_completo)
    if response.status_code == 200:
        print('Requisição realizada com sucesso!')
    response.raise_for_status()

    dados_ipca = json.loads(response.content)
    indices_ipca = dados_ipca[0]['resultados'][0].get('series')[0].get('serie')
    print(f'{len(indices_ipca)} índices IPCA mensais extraídos do IBGE')

    # Cria uma lista de tuplas a partir dos itens do dicionário
    lista_dados = list(indices_ipca.items())

    # Cria o DataFrame
    df = pd.DataFrame(lista_dados, columns=["AnoMes", "Valor"])    

    # 1. Converter a coluna `Valor` para o tipo numérico
    df['Valor'] = pd.to_numeric(df['Valor'])

    # # Filtrar o DataFrame para o período desejado
    # df = df[
    #     (int(str(df['AnoMes'])[:4]) >= ano_inicio) & (int(str(df['AnoMes'])[:4]) <= ano_final)
    # ]

    # 2. Criar uma nova coluna `Ano` extraindo o ano da coluna `AnoMes`
    df['Ano'] = df['AnoMes'].astype(str).str[:4].astype(int)

    # 3. Calcular a variação percentual mensal do IPCA
    df['InflacaoMensal'] = df['Valor'].pct_change() * -100
    df['InflacaoMensal'] = df['InflacaoMensal'].fillna(0)

    # 4. Calcular a inflação acumulada por ano
    df_inflacao_anual = df.groupby('Ano')['InflacaoMensal'].sum().reset_index()
    df_inflacao_anual = df_inflacao_anual.rename(columns={'InflacaoMensal': 'PerdaInflacao'})

    # 5. Criar um dicionário com anos e correções
    anos = df_inflacao_anual['Ano'].unique().tolist()
    correcoes_salariais_lista = [0 for x in anos]  # Lista de zeros por padrão
    if reajustes_anuais:  # Verificar se a lista de reajustes não está vazia
        correcoes_salariais_lista = reajustes_anuais[:len(anos)]  # Usar os reajustes fornecidos
    correcoes_salariais = dict(zip(anos, correcoes_salariais_lista))

    # 6. Criar um DataFrame a partir das correções salariais
    df_salarios = pd.DataFrame(list(correcoes_salariais.items()), columns=['Ano', 'CorrecaoSalarial'])

    # 7. Fazer o merge entre os DataFrames
    df_merged = df_inflacao_anual.merge(df_salarios, on='Ano', how='left')

    # 8. Preencher os valores NaN da coluna 'CorrecaoSalarial' com 0
    df_merged['CorrecaoSalarial'] = df_merged['CorrecaoSalarial'].fillna(0)

    # 9. Calcular a diferença acumulada entre a correção salarial e a perda por inflação
    df_merged['SaldoAcumulado'] = (df_merged['CorrecaoSalarial'] - df_merged['PerdaInflacao']).cumsum()

    # Filtrar o DataFrame para o período desejado
    df_merged_filtrado = df_merged[
        (df_merged['Ano'] >= ano_inicio) & (df_merged['Ano'] <= ano_final)
    ]

    # Zerar o saldo acumulado no ano de início
    df_merged_filtrado.loc[df_merged_filtrado['Ano'] == ano_inicio, 'SaldoAcumuladoAjustado'] = 0

    # 10. Calcular o valor absoluto máximo para definir o domínio do eixo y principal e secundário
    max_abs_valor = max(
        df_merged_filtrado['PerdaInflacao'].abs().max(),
        df_merged_filtrado['CorrecaoSalarial'].abs().max(),
        df_merged_filtrado['SaldoAcumulado'].abs().max()
    )

    # Preparar os dados para o gráfico de barras
    df_barras = df_merged_filtrado.melt(id_vars='Ano', value_vars=['PerdaInflacao', 'CorrecaoSalarial'], var_name='Tipo', value_name='Valor')

    # Calcular o valor absoluto máximo para definir o domínio do eixo y principal
    max_abs_valor_barras = max(df_merged_filtrado['PerdaInflacao'].abs().max(), df_merged_filtrado['CorrecaoSalarial'].abs().max())

    # Calcular o valor mínimo e máximo do SaldoAcumulado para definir o domínio do eixo y da linha
    min_saldo_acumulado = df_merged_filtrado['SaldoAcumulado'].min()
    max_saldo_acumulado = df_merged_filtrado['SaldoAcumulado'].max()

    # Criar o gráfico de barras
    barras = alt.Chart(df_barras).mark_bar().encode(
        x=alt.X('Ano:O', axis=alt.Axis(labelAngle=-45)),
        y=alt.Y('Valor:Q', axis=alt.Axis(title='Variação (%)', titleColor='black'), scale=alt.Scale(domain=[-max_abs_valor_barras, max_abs_valor_barras])),
        color=alt.Color('Tipo:N', scale={'domain': ['PerdaInflacao', 'CorrecaoSalarial'], 'range': ['red', 'steelblue']}),
        tooltip=['Ano', 'Valor', 'Tipo']
    ).properties(
        title='Perda por Inflação, Reposição Salarial e Saldo Acumulado'
    )

    # Adicionar os rótulos às barras
    text = barras.mark_text(
        align='center',
        baseline='bottom',
        dy=-5
    ).encode(
        text=alt.Text('Valor:Q', format='.2f')
    )

    # Criar o gráfico de linha para o saldo acumulado (eixo secundário)
    linha = alt.Chart(df_merged_filtrado).mark_line(color='green', point=True).encode(
        x='Ano:O',
        y=alt.Y('SaldoAcumulado:Q', axis=alt.Axis(title='Saldo Acumulado (%)', titleColor='green'), scale=alt.Scale(domain=[min_saldo_acumulado, max_saldo_acumulado])),
        tooltip=['Ano', 'SaldoAcumulado']
    )

    # Adicionar rótulos de dados à linha
    text_linha = linha.mark_text(
        align='left',
        baseline='middle',
        dx=5,  # Nudge text to right so it doesn't overlap with the line
        color='black'
    ).encode(
        text=alt.Text('SaldoAcumulado:Q', format='.2f')
    )

    # Combinar os gráficos com os rótulos
    grafico_final = alt.layer(barras + text, linha + text_linha).resolve_scale(
        y='independent'
    ).interactive()


    grafico_final = grafico_final.properties(
        height=600,  # Largura de 800 pixels
        width=1100  # Largura de 1200 pixels
    )

    return df_merged, grafico_final

# Exemplo de uso
ano_inicio = 2003
ano_final = 2023
reajustes_anuais = [2.5, 3.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5, 10.0, 10.5, 11.0, 11.5, 12.0, 12.5, 13.0, 13.5]  # Exemplo de reajustes

df_merged, grafico_final = plotar_perdas(ano_inicio, ano_final, reajustes_anuais)
grafico_final

In [ ]:
import altair as alt
alt.renderers.enable('jupyterlab')
grafico_final

In [ ]:
# Converter a coluna `Valor` para o tipo numérico
df['Valor'] = pd.to_numeric(df['Valor'])

# Extrair o ano da coluna `AnoMes`
df['Ano'] = df['AnoMes'].astype(str).str[:4].astype(int)

# Filtrar para manter apenas os meses de dezembro
df_dezembro = df[df['AnoMes'].astype(str).str[-2:] == '12'].copy()

# Agrupar por ano e selecionar o primeiro valor de cada ano
df_anual = df_dezembro.groupby('Ano')['Valor'].first().reset_index()

# Obter o ano de início e o ano de término do usuário
ano_inicio = int(input("Digite o ano de início: "))
ano_termino = int(input("Digite o ano de término: "))

# Filtrar para o período desejado
df_periodo = df_anual[(df_anual['Ano'] >= ano_inicio) & (df_anual['Ano'] <= ano_termino)]

# Calcular a variação acumulada do IPCA
variacao_acumulada = (df_periodo['Valor'].iloc[-1] / df_periodo['Valor'].iloc[0] - 1) * 100

# Calcular a taxa anual equivalente
num_anos = ano_termino - ano_inicio + 1
taxa_anual = ((1 + variacao_acumulada / 100) ** (1 / num_anos) - 1) * 100

# Imprimir os resultados
print(f'Variação acumulada do IPCA de {ano_inicio} a {ano_termino}: {variacao_acumulada:.2f}%')
print(f'Taxa anual equivalente: {taxa_anual:.2f}%')

# Filtrar para o período desejado
df_periodo = df_anual[(df_anual['Ano'] >= ano_inicio) & (df_anual['Ano'] <= ano_termino)]

# Calcular a variação acumulada do IPCA
variacao_acumulada = (df_periodo['Valor'].iloc[-1] / df_periodo['Valor'].iloc[0] - 1) * 100

# Calcular a taxa anual equivalente
num_anos = ano_termino - ano_inicio + 1
taxa_anual = ((1 + variacao_acumulada / 100) ** (1 / num_anos) - 1) * 100

# Imprimir os resultados
print(f'Variação acumulada do IPCA de {ano_inicio} a {ano_termino}: {variacao_acumulada:.2f}%')
print(f'Taxa anual equivalente: {taxa_anual:.2f}%')

In [ ]:
# Converter a coluna `Valor` para o tipo numérico
df['Valor'] = pd.to_numeric(df['Valor'])

# Extrair o ano da coluna `AnoMes`
df['Ano'] = df['AnoMes'].astype(str).str[:4].astype(int)

# Filtrar para manter apenas os meses de dezembro
df_dezembro = df[df['AnoMes'].astype(str).str[-2:] == '12'].copy()

# Agrupar por ano e selecionar o primeiro valor de cada ano
df_anual = df_dezembro.groupby('Ano')['Valor'].first().reset_index()

# Obter o ano de início e o ano de término do usuário
ano_inicio = int(input("Digite o ano de início: "))
ano_termino = int(input("Digite o ano de término: "))

# Filtrar para o período desejado
df_periodo = df_anual[(df_anual['Ano'] >= ano_inicio) & (df_anual['Ano'] <= ano_termino)]

# Calcular a variação acumulada do IPCA
variacao_acumulada = (df_periodo['Valor'].iloc[-1] / df_periodo['Valor'].iloc[0] - 1) * 100

# Calcular a taxa anual equivalente
num_anos = ano_termino - ano_inicio + 1
taxa_anual = ((1 + variacao_acumulada / 100) ** (1 / num_anos) - 1) * 100

# Imprimir os resultados
print(f'Variação acumulada do IPCA de {ano_inicio} a {ano_termino}: {variacao_acumulada:.2f}%')
print(f'Taxa anual equivalente: {taxa_anual:.2f}%')

In [ ]:
import altair as alt

# Criar um DataFrame com os dados de reposição salarial
df_reposicao = df_merged[['Ano', 'CorrecaoSalarial']].rename(columns={'CorrecaoSalarial': 'Valor'})
df_reposicao['Tipo'] = 'Reposição Salarial'

# Criar um DataFrame com os dados de perda por inflação
df_perda_inflacao = df_merged[['Ano', 'InflacaoPercentual']].rename(columns={'InflacaoPercentual': 'Valor'})
df_perda_inflacao['Valor'] = -df_perda_inflacao['Valor']  # Inverter os valores para ficarem negativos
df_perda_inflacao['Tipo'] = 'Perda por Inflação'

# Concatenar os DataFrames de reposição e perda
df_plot = pd.concat([df_reposicao, df_perda_inflacao])

# Calcular o valor absoluto máximo para definir o domínio do eixo y principal
max_abs_valor = df_plot['Valor'].abs().max()

# Criar o gráfico de barras
barras = alt.Chart(df_plot).mark_bar().encode(
    x=alt.X('Ano:O', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('Valor:Q', scale=alt.Scale(domain=[-max_abs_valor, max_abs_valor])),  # Ajustar escala y
    color=alt.Color('Tipo:N', scale={'domain': ['Perda por Inflação', 'Reposição Salarial'], 'range': ['red', 'steelblue']}),
    tooltip=['Ano', 'Valor', 'Tipo']
).properties(
    title='Reposição Salarial, Perda por Inflação e Saldo Acumulado'
)

# Adicionar os rótulos às barras
text = barras.mark_text(
    align='center',
    baseline='bottom',
    dy=-5  # Nudge text upwards so it doesn't overlap with the bar
).encode(
    text=alt.Text('Valor:Q', format='.2f')
)

# Criar o gráfico de linha para o saldo acumulado (eixo secundário)
linha = alt.Chart(df_merged).mark_line(color='green', point=True).encode(
    x='Ano:O',
    y=alt.Y('DiferencaAcumulada:Q', axis=alt.Axis(title='Saldo Acumulado', titleColor='green')),
    tooltip=['Ano', 'DiferencaAcumulada']
)

# Combinar os gráficos
grafico_final = alt.layer(barras + text, linha).resolve_scale(
    y = 'independent'
).interactive()

# Exibir o gráfico
grafico_final

grafico_final = grafico_final.properties(
    height=600,  # Largura de 800 pixels
    width=1100  # Largura de 1200 pixels
)
# Salve o gráfico em um arquivo JSON
grafico_final.save('grafico_perda_ganho_poder_compra.json')

In [ ]:
import altair as alt
alt.renderers.enable('jupyterlab')
grafico_final

In [ ]:
df_merged

In [ ]:
from matplotlib import pyplot as plt

plt.plot(indices_ipca.values())

In [ ]:
len(dados_ipca[0])

In [ ]:
import requests
import pandas as pd
import json
import csv
from io import BytesIO, TextIOWrapper

url_tab_var = "https://apisidra.ibge.gov.br/values/t/1737/n1/all/v/2266/p/all"
indices_mensais = {}

response = requests.get(url_tab_var)
response.raise_for_status()

meses=[]
indices=[]

# Ler o CSV com TextIOWrapper para converter bytes em strings
with TextIOWrapper(BytesIO(response.content), encoding="utf-8") as f:
    reader = csv.reader(f, delimiter=";")

    # Extrair dados relevantes e construir dicionário
    for row in reader:
        print(row)
        if "D3C" in row:
            meses.append(row)
        if "V" in row:
            indices.append(row)

In [ ]:
len(meses)

In [ ]:
len(indices)

In [ ]:
# !pip install sidrapy

In [ ]:
import sidrapy

# Importa os dados do SIDRA
ipca_raw = sidrapy.get_table(table_code= "1737",
                            territorial_level = "1",
                            ibge_territorial_code = "all",
                            period = "all",
                            )

# Checamos a importação
ipca_raw.head()

In [ ]:
import pandas as pd

# Converter a coluna D3C para numérico, ignorando valores não numéricos
ipca_raw['D3C'] = pd.to_numeric(ipca_raw['D3C'], errors='coerce')

# Filtrar as linhas onde D3C é igual a 2265
filtro = ipca_raw['D3C'] == 2265
ipca_filtrado = ipca_raw[filtro]

ipca_filtrado[["V","D2C"]]

In [ ]:
import sidrapy

# Importa os dados do SIDRA
pib_sa_raw = sidrapy.get_table(table_code= "1621",
                            territorial_level = "1",
                            ibge_territorial_code = "all",
                            period = "all",
                            classification = "11255/90707")

# Checamos a importação
pib_sa_raw.head()

In [ ]:
from poder_compra import IPCAData

ipca = IPCAData()
meses, indices = ipca.obter_numeros_indice()
for i, j in zip(meses, indices):
    print(i,j)


In [ ]:
indices[:3]

In [ ]:
meses[0]

In [ ]:
import json
a,b = indices[3].split(": ")
{a.replace("\"",""): (b.replace("\"","").replace(",",""))}

In [ ]:
a.replace("\"","")

In [ ]:
float(b.replace("\"","").replace(",",""))

In [ ]:
# Extrair dados relevantes e construir dicionário
for row in reader:
    if row[2] == "MN" and row[1] == "Número-índice":
        data = row[4270].replace(" ", "-") 
        valor = float(row[3].replace(",", "."))
        indices_mensais[data] = valor

In [ ]:
indices_mensais

In [ ]:
from poder_compra import IPCAData

ipca_data = IPCAData()
poder_compra = PoderCompra(ipca_data.dados_ipca)
perdas = poder_compra.calcular_perdas()

previsao = PrevisaoInflacao(ipca_data.dados_ipca)
inflacao_futura = previsao.prever()

Visualizacao.plotar_evolucao_poder_compra(ipca_data.dados_ipca, perdas)